In [3]:
# -- coding: utf-8 --
import requests, json, datetime, boto3, itertools, tweepy
from multiprocessing.dummy import Pool as ThreadPool
import pyspark.sql.functions as F
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.appName('twitter_crawler').getOrCreate()
sc = spark.sparkContext

In [6]:
# twitter keys
CONSUMER_KEY = 'yXKMKyVm0YmkzMw6e9zAKg2U3'
CONSUMER_SECRET = 'kjmviFnWNPocmBy0xtFxndQ17oKluLM5Ghgu7IQoVUix4HHs3K'
ACCESS_TOKEN = 	'1567104799-zC1CJwjfYWiPAOI9325WLZWe3Z5xyIVdvX4338m'
ACCESS_SECRET = '2U9iV3xy5kFVojA7pwFxkPxasSCZ2qs9hT6asavdSPzdg'

In [7]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

In [11]:
def get_tweets(twitter_username):
    tweets = api.user_timeline(screen_name = twitter_username, count = 100, include_rts = True, tweet_mode = 'extended')

    tweets = [tweet._json for tweet in tweets]
    tweets = list(map(lambda x: {
        'tweet' : x['full_text'],
        'dt_creation' : x['created_at'],
        'retweet_count' : x['retweet_count'],
        'favorite_count' : x['favorite_count'],
        'possibly_sensitive' : x['possibly_sensitive']
        'dt_extraction' : datetime.datetime.now()
    }, tweets))

    return tweets

In [ ]:
politicians_twitter_df = spark.read.parquet("/FileStore/tables/curated/politicians_twitter/*.parquet").cache()

In [ ]:
get_tweets_udf = F.udf(lambda twitter_username: get_tweets(twitter_username), ArrayType())

politicians_twitter_df = politicians_twitter_df.withColumn('tweets', get_tweets_udf('DC_TWITTER_USERNAME'))

In [12]:
tweets = get_tweets('CarlosBolsonaro')

In [14]:
tweets[0]

{'created_at': 'Sat May 09 18:00:37 +0000 2020',
 'id': 1259181496289710082,
 'id_str': '1259181496289710082',
 'full_text': 'https://t.co/zqGP3cjErj',
 'truncated': False,
 'display_text_range': [0, 0],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1259181484080009216,
    'id_str': '1259181484080009216',
    'indices': [0, 23],
    'media_url': 'http://pbs.twimg.com/media/EXmC42pWkAAccml.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/EXmC42pWkAAccml.jpg',
    'url': 'https://t.co/zqGP3cjErj',
    'display_url': 'pic.twitter.com/zqGP3cjErj',
    'expanded_url': 'https://twitter.com/CarlosBolsonaro/status/1259181496289710082/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 1200, 'h': 1200, 'resize': 'fit'},
     'small': {'w': 680, 'h': 680, 'resize': 'fit'},
     'large': {'w': 2048, 'h': 2048, 'resize': 'fit'}}}]},
 'extended_entities': {'media': [{'id